# (1) Importing/Storing data

In [1]:
import wfdb
import os
import glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
records_A = []
properties_A = []
annot_A = []
AnnSymb_A = []
AnnSamp_A = []
AnnRhythm_A = []
Rpeak_Samp_A = []
Rpeak_Symb_A = []


for f in glob.glob('C:/Users/masud/Skrivebord/Github_Bachelor_Project/wfdb-python/data/arrhythmia_data/*.dat'): ##### change the path to own directory.       
    sig, fields = wfdb.rdsamp(f[:-4], channels=[1])  #### In this function, pass "channels=[0]" or "channels=[1]" to select channel 1 or 2.
    
    ann = wfdb.rdann(f[:-4], 'atr')
    QRS = wfdb.rdann(f[:-4], 'xws')
    Symb = pd.Series(ann.symbol)
    Samp = pd.Series(ann.sample)
    
    QRS_Symb = pd.Series(QRS.symbol)
    QRS_Samp = pd.Series(QRS.sample)
    Rhythm = pd.Series(ann.aux_note)
    records_A.append(sig)
    properties_A.append(fields)
    annot_A.append(ann)
    AnnSymb_A.append(Symb)
    AnnSamp_A.append(Samp)
    Rpeak_Symb_A.append(QRS_Symb)
    Rpeak_Samp_A.append(QRS_Samp)
    AnnRhythm_A.append(Rhythm)
    
    

AnnSymb_A = pd.Series(AnnSymb_A).values
AnnSamp_A = pd.Series(AnnSamp_A).values

In [3]:
# AnnSymb_A[24] #Rpeak_Symbol

In [4]:
# AnnSamp_A[24] #Rpeak_Sample

In [5]:
# Rpeak_Symb_A[47] ###Annotaion_Symbol
# print(len(Rpeak_Symb_A[47])) ### length = 43

In [6]:
# Rpeak_Samp_A[24] ### Annotaion_Sample
# print(len(Rpeak_Samp_A[47])) ### length = 43

In [7]:
# AnnRhythm_A[24] ### length = 2764

# (2) Pre processing data

In [8]:
len(records_A[0])

650000

In [9]:
records_A[0]

array([[-0.065],
       [-0.065],
       [-0.065],
       ...,
       [-0.365],
       [-0.335],
       [ 0.   ]])

In [10]:
len(AnnSymb_A[0])

2274

In [11]:
len(AnnRhythm_A[0])

2274

In [12]:
AnnRhythm_A[0]

0       (N 
1          
2          
3          
4          
       ... 
2269       
2270       
2271       
2272       
2273       
Length: 2274, dtype: object

In [13]:
# lst = []
# for i in range(48):
#     print(f"Length of record {i} is: {len(AnnSymb_A[i])}")
#     for item in range(len(AnnSymb_A[i])):
#         lst.append(item)
# print("Total amount of r-peaks: ", len(lst))

In [14]:
appended_AnnSamp = [item for i in range(len(AnnSamp_A)) for item in AnnSamp_A[i]]
appended_AnnSymb = [item for i in range(len(AnnRhythm_A)) for item in AnnRhythm_A[i]]

In [15]:
df = pd.DataFrame(
    {'Rpeaks': appended_AnnSamp,
     'Label': appended_AnnSymb
    })

df

,Rpeaks,Label
0,18,(N 
1,77,
2,370,
3,662,
4,946,
...,...,...
112642,648797,
112643,649040,
112644,649292,
112645,649536,


### Rescaling 360 Hz samples (Arrhythmia) into 250 Hz samples (Atrial Fibrillation)

In [16]:
df['Rpeaks'] = round(0.694 * df['Rpeaks'])

### Adding differences column

In [17]:
df['RRI'] = abs(df['Rpeaks'] - df['Rpeaks'].shift(-1))
df

,Rpeaks,Label,RRI
0,12.0,(N ,41.0
1,53.0,,204.0
2,257.0,,202.0
3,459.0,,198.0
4,657.0,,197.0
...,...,...,...
112642,450265.0,,169.0
112643,450434.0,,175.0
112644,450609.0,,169.0
112645,450778.0,,164.0


### Labeling all the samples

In [18]:
df = df.replace(r'^\s*$', np.nan, regex=True) ### Replace empty places with NaN
df

,Rpeaks,Label,RRI
0,12.0,(N ,41.0
1,53.0,NaN,204.0
2,257.0,NaN,202.0
3,459.0,NaN,198.0
4,657.0,NaN,197.0
...,...,...,...
112642,450265.0,NaN,169.0
112643,450434.0,NaN,175.0
112644,450609.0,NaN,169.0
112645,450778.0,NaN,164.0


In [19]:
df = df.fillna(method='ffill') ### Perform "forward fill" - filling instances (NaN) in front row with preceeeding value
df

,Rpeaks,Label,RRI
0,12.0,(N ,41.0
1,53.0,(N ,204.0
2,257.0,(N ,202.0
3,459.0,(N ,198.0
4,657.0,(N ,197.0
...,...,...,...
112642,450265.0,(N ,169.0
112643,450434.0,(N ,175.0
112644,450609.0,(N ,169.0
112645,450778.0,(N ,164.0


In [20]:
# unique, counts = np.unique(df.Label, return_counts=True)
# dict(zip(unique, counts))

In [21]:
# final_df = df[(df['Label'] == '(N') | (df['Label'] == '(AFIB')]
# final_df

arrhythmia_list = ['(N', '(N\x00', '(AFIB', '(AFIB\x00']

final_df = df[df['Label'].isin(arrhythmia_list)]
final_df

,Rpeaks,Label,RRI
0,12.0,(N ,41.0
1,53.0,(N ,204.0
2,257.0,(N ,202.0
3,459.0,(N ,198.0
4,657.0,(N ,197.0
...,...,...,...
112642,450265.0,(N ,169.0
112643,450434.0,(N ,175.0
112644,450609.0,(N ,169.0
112645,450778.0,(N ,164.0


In [22]:
### Change (N\x00 and (AFIB\x00
final_df.loc[df['Label'].astype(str) == '(N\x00', 'Label'] = '(N'
final_df.loc[df['Label'].astype(str) == '(AFIB\x00', 'Label'] = '(AFIB'
final_df

,Rpeaks,Label,RRI
0,12.0,(N,41.0
1,53.0,(N,204.0
2,257.0,(N,202.0
3,459.0,(N,198.0
4,657.0,(N,197.0
...,...,...,...
112642,450265.0,(N,169.0
112643,450434.0,(N,175.0
112644,450609.0,(N,169.0
112645,450778.0,(N,164.0


In [23]:
unique2, counts2 = np.unique(final_df.Label, return_counts=True)
dict(zip(unique2, counts2))

{'(AFIB': 11675, '(N': 79412}

In [24]:
# final_df['Label'] = np.where((final_df.Label == '(N\x00'), '(N', final_df.Label)
# final_df

In [25]:
# final_df.Label == '(N\x00'
# mask = final_df.Label == '(N\x00'
# column_name = 'Label'
# df.loc[mask, column_name] = '(N'

***Some major diviations in samples calculates differences with values of approx. 40.000 
where it should be approx. 200-300, this rows are omitted***

In [26]:
final_df = final_df.drop(final_df[final_df.RRI > 1000].index)
final_df

,Rpeaks,Label,RRI
0,12.0,(N,41.0
1,53.0,(N,204.0
2,257.0,(N,202.0
3,459.0,(N,198.0
4,657.0,(N,197.0
...,...,...,...
112642,450265.0,(N,169.0
112643,450434.0,(N,175.0
112644,450609.0,(N,169.0
112645,450778.0,(N,164.0


In [27]:
#### For printing/viewing entire Dataframe
# with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
#     print(final_df)

### Convert 'Label' column to boolean expr ----> '(AFIB' = 1 and '(N' = 0

In [28]:
final_df['Label'] = final_df['Label'].map( 
                   {'(N':False , '(AFIB':True}) 
# final_df['Label'] = final_df['Label'].map( 
#                    {'(N\x00':False ,'(AFIB\x00':True})

In [29]:
final_df['Label'] = final_df['Label'].astype(int)
final_df

,Rpeaks,Label,RRI
0,12.0,0,41.0
1,53.0,0,204.0
2,257.0,0,202.0
3,459.0,0,198.0
4,657.0,0,197.0
...,...,...,...
112642,450265.0,0,169.0
112643,450434.0,0,175.0
112644,450609.0,0,169.0
112645,450778.0,0,164.0


In [30]:
unique2, counts2 = np.unique(final_df.Label, return_counts=True)
dict(zip(unique2, counts2))

{0: 79379, 1: 11670}

In [31]:
def segmenting_record_A(seg_value_A):
    rri2_A = final_df['RRI'] ### Storing the intervals between rpeaks
    rri2_A = np.array(rri2_A, dtype=np.float64) ### Ensuring no overflow issues happens, when calculating in for loop later
    
    amount_A = -(len(rri2_A) % seg_value_A) # amount of data points to remove, for equal length segments with no residue points
    print(f"Amount to remove {amount_A}")
    
    rec_amount_A = rri2_A[:amount_A]
    seg_shape_A = len(rec_amount_A) // seg_value_A # amount of total segments (given the specified segment length) 
    print(f"Shape 0: {seg_shape_A}")
    
    segmented_rec_A = rec_amount_A.reshape(seg_shape_A,seg_value_A)
    return segmented_rec_A

In [32]:
segmented_record_A = segmenting_record_A(10) # Specify the segment length

Amount to remove -29
Shape 0: 1517


### X

In [33]:
### Loads in functions for Shannon Entropy, Mean absolute deviation calculations
%run "features_utils.ipynb"
from scipy.stats import median_abs_deviation

In [34]:
all_features_A = []

### Calculating features for every single segmented "block" inside the segmented_record_A variable
for x in range(len(segmented_record_A)):
    
    #MEAN
    ff1 = np.nanmean(segmented_record_A[x]) ### nanmean, nanstd computes values while ignoring nan-values
    #STD
    ff2 = np.nanstd(segmented_record_A[x])
    #RMSSD
    sum_ = 0
    for y in range(len(segmented_record_A[x]) - 1): ### loops 19 times
        sum_ += (segmented_record_A[x][y] - segmented_record_A[x][y+1])**2
    sum_multiplied = 1/(len(segmented_record_A) - 1) * sum_
    ff3 = np.sqrt(sum_multiplied)
    #NORMALIZED RMSSD
    ff4 = (ff3 / ff1)
    #SHANNON ENTROPY
    ff5 = entropy(segmented_record_A[x])
    #MEAN ABSOLUTE DEVIATION
    ff6 = mean_abs_deviation(segmented_record_A[x])
    #MEDIAN ABSOLUTE DEVIATION
    ff7 = median_abs_deviation(segmented_record_A[x])
    
    #COVARIANCE
    #z = np.stack((ff1, ff2))
    #ff5 = np.cov(z) ### try cov with previous segment

    my_features = pd.Series([np.around(ff1,3), np.around(ff2,3), np.around(ff3,3), np.around(ff4,3), np.around(ff5,3), 
                             np.around(ff6,3), np.around(ff7,3)],
                            index=['Mean','STD','RMSSD','Normalized RMSSD','Shannon Entropy',
                                   'Mean absolute deviation','Median absolute deiviation'])
    all_features_A.append(my_features)

In [35]:
n = 0

print(f"Features calculated for an example segment:\n{all_features_A[n]}") 
print('\n'f"Features variable contains {len(all_features_A)} segments")

Features calculated for an example segment:
Mean                          200.567
STD                            22.949
RMSSD                           5.093
Normalized RMSSD                0.025
Shannon Entropy                 0.741
Mean absolute deviation         9.220
Median absolute deiviation      5.000
dtype: float64

Features variable contains 1517 segments


### Y

### Paste the copied information below (indicated with the arrows)
1. Arrow: Add "Amount to remove", (remember the negative sign).
2. Arrow: Add "Shape 0" as first argument and the segment size as second argument. (Shape 0 , segment size)
3. Arrow: Divide segment size by 2 and add here. (meaning, if over half the segment contains "(AFIB", than classify the whole segment as "(AFIB"

In [37]:
Label_A = final_df["Label"]
Label_A = np.array(Label_A, dtype=np.float64)

y_amount_A = Label_A[:-9] # <=============== (1)
y_shape_A = y_amount_A.reshape(9104, 10) # <=============== (2)
y_list_A = []

### Calc if every 20 segment block is Normal synus rythm or AFIB
for g in range(len(y_shape_A)):
    #y_segment = y_shape
    sum_segment_A = np.sum(y_shape_A[g])
    if sum_segment_A >= 5: # <=============== (3)
        sum_segment_A = 1
    else: sum_segment_A = 0
    y_list_A.append(sum_segment_A)

y_list_A = np.array(y_list_A, dtype=np.float64)
print(y_list_A.dtype)
print(f"Length of y_list: {len(y_list_A)}")

float64
Length of y_list: 1517
